# Table Pre-training with `TapasForMaskedLM`



**Author**: [Yookyung Kho](https://github.com/yookyungkho)

**Date presented**: 2022/07/25, DSBA keras2torch Study

**Task description**: Table-aware Masked Language Model

   - Pre-Training Bert-based Tapas Model with KorWikiTabular dataset for Korean Table MRC(ex. QA)


**References**:

- https://huggingface.co/docs/transformers/model_doc/tapas

- https://github.com/huggingface/transformers/blob/v4.11.3/src/transformers/models/tapas/tokenization_tapas.py

- https://github.com/huggingface/transformers/blob/v4.11.3/src/transformers/data/data_collator.py

- https://github.com/hwk0702/keras2torch/blob/main/Natural_Language_Processing/Question_Answering_Huggingface/QA_huggingface_KYK.ipynb


---

## 📜Prerequisite: About Table QA

<img src="imgs/table_qa_in_tapas.PNG" width="900" height="600">

Table Question Answering이란, 주어진 query(질문)에 대한 정답을 질문과 매핑되어 있는 table에서 찾아내는 task를 일컫습니다.

Table QA를 잘 풀기 위해서는 text와 table의 정보를 잘 함축한 joint representation을 학습해야 합니다.

이를 위해서 아래와 같이 Downstream task인 Table QA를 풀기 이전에 대량의 table-text 데이터를 가지고 사전학습을 진행합니다.

<img src="imgs/tapas_pretraining.PNG" width="900" height="500">


<br/>

기존 table 연구 흐름을 살펴보면, 대표적으로 google research에서 table을 text와 함께 인코딩하는 방식에 대해 다양한 연구를 진행했습니다.

이 노트북에서 다루게 될 모델인 Tapas 역시 google에서 발표한 구조입니다. ([paper](https://arxiv.org/pdf/2004.02349), [github](https://github.com/google-research/tapas))

google은 자사의 github 페이지에 tensorflow로 구현된 코드를 공개하여 pytorch 유저의 입장에서는 활용도가 높지 않았는데, 얼마 전 huggingface 플랫폼을 통해 Tapas의 Pytorch 구현체가 공개되었습니다.(야호!😁)

오늘은 Table을 Text와 함께 인코딩하는 가장 기본적인 방법론을 다루고 있는 TAPAS에 대해 살펴보겠습니다!

### 📦Dataset

2022년 드디어! LG AI Research에서 한국어 버전의 Table MRC 데이터셋을 공개하였습니다!

데이터셋과 함께 공개한 paper는 LREC 2022에 accept되었습니다.([Full paper 보러 가기](https://arxiv.org/abs/2201.06223))

<img src="imgs/table_mrc.PNG" width="900" height="450">

공개된 데이터셋은 Pre-training(사전학습)용 데이터인 `KorWikiTabular`과 Fine-tuning(QA)용 데이터 `KorWikiTQ`, 이렇게 2가지로 나뉩니다.


- 참고로, 데이터는 [LG AI Research Github](https://github.com/LG-NLP)의 [KorWikiTableQuestions Repository](https://github.com/LG-NLP/KorWikiTableQuestions)에서 다운받으실 수 있습니다.

---

## 0. 준비 과정

### 0.1. 데이터 샘플링

In [1]:
import json

file_path = "data/KorWikiTabular.json"

with open(file_path, 'r', encoding="UTF-8") as f:
    tables = json.loads(f.read())

In [2]:
len(tables['data'])

1196306

앞서 소개한 한국어 table 사전학습 데이터(`KorWikiTabular`)는 무려 약 **120만 개**의 샘플을 포함하고 있습니다.😱

따라서, 모든 데이터를 가지고 모델링을 진행하기에는 무리가 있어 보입니다.(_GPU 메모리도 부족할 뿐더러, 시간이 너무 오래 걸리겠죠?_)

원활한 실험을 위해 **200개의 샘플만 랜덤으로 추출**하여 `sample_200_KorWikiTabular.json`의 형태로 `data` 경로에 저장하겠습니다.

In [ ]:
import numpy as np

np.random.seed(602)

randn_idxs = np.random.choice(list(range(len(tables['data']))), size=200, replace=False) #비복원추출

sample_tables = {'data': []}

for new_id, org_id in enumerate(randn_idxs):
    sample_tables['data'].append(tables['data'][org_id])
    sample_tables['data'][new_id]['org_idx'] = int(org_id)
    
file_path = "data/sample_200_KorWikiTabular.json"
json.dump(sample_tables, open(file_path,'w'), indent=4)

In [ ]:
len(sample_tables['data'])

저장 완료되었습니다!

앞으로는 위 4개 cell 실행 없이, sample data만 바로 불러들여와서 실험 진행하도록 하겠습니다.

---

In [1]:
import json

file_path = "data/sample_200_KorWikiTabular.json"

with open(file_path, "r") as json_file:
    sample_data = json.load(json_file)

In [2]:
len(sample_data['data'])

200

### 0.2. Module Import, GPU 세팅

지금처럼 소량(200개)의 샘플 데이터만 뽑아 쓰는 경우가 아니라 Full dataset을 모두 돌릴 때에는, local gpu만으로 MLM 학습이 불가능합니다.(이유: 메모리 부족)

따라서, multi-gpu를 장착한 서버 환경에서 돌려야 하는데, 이 경우에 필요한 코드도 함께 적어두었으니 참고해주세요!



> 💙Requirements
> 
> - `torch == 1.12.0+cu113`
> - `torch-scatter == 2.0.9`(for `TapasModel`)
> - `transformers == 4.11.3`
> - `pandas == 1.3.5`

- 이 중 `torch-scatter` 설치 관련해서는 [이 link](https://github.com/rusty1s/pytorch_scatter)를 참고해주세요.

In [3]:
import wget
import os
import random
import json
import argparse
import wandb

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn

from transformers import TapasConfig, TapasTokenizer, TapasForMaskedLM, AdamW, get_scheduler

In [4]:
# multi gpu

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1,2,3"  # Set the GPUs to use

In [5]:
parser = argparse.ArgumentParser()

parser.add_argument("--vocab_model_name", default="klue/bert-base", type=str)
parser.add_argument("--tok_path", default="table_tokenizer", type=str)

parser.add_argument("--max_seq_len", default=512, type=int)
parser.add_argument("--max_query_len", default=470, type=int)
parser.add_argument("--row_del_ratio", default=0.9, type=float)
parser.add_argument("--col_del_ratio", default=0.9, type=float)
parser.add_argument("--mlm_prob", default=0.15, type=float)

parser.add_argument("--random_seed", default=602, type=int)

parser.add_argument("--epoch", default=5, type=int)
parser.add_argument("--batch_size", default=16, type=int)
parser.add_argument("--learning_rate", default=4e-4, type=float)
parser.add_argument("--weight_decay", default=0.0, type=float)
parser.add_argument("--lr_scheduler_type", default="linear", type=str)
parser.add_argument("--num_warmup_steps", default=0, type=int)
parser.add_argument("--eval_step", default=10, type=int)

parser.add_argument("--wandb_project", default="Table Pretraining", type=str)
parser.add_argument("--wandb_name", default="tapas-base-mlm-clean", type=str)
parser.add_argument("--wandb_entity", default="yookyungkho", type=str)

parser.add_argument("--output_dir", default="models/", type=str)

args = parser.parse_args([])

In [6]:
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #multi_gpu

print('Device:', args.device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

if torch.cuda.is_available():
    n_gpu = torch.cuda.device_count()
    print("number of gpu: ", n_gpu)

Device: cuda
Current cuda device: 0
Count of using GPUs: 4
number of gpu:  4


In [7]:
# seed 고정
def seed_everything(seed):
    # random.seed(seed) #masking dynamics를 위해 이 부분은 주석 처리
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(args.random_seed)

---

이제 모델링을 위한 준비를 마쳤습니다.

지금부터는 본격적으로 사전학습을 위한 데이터셋을 전처리하고 모델을 구축한 뒤, 실제 학습을 진행하도록 하겠습니다.

렛츠공~💛

---

## 1. 데이터 전처리

Tapas의 사전학습 task는 Table-aware Masked Language Model입니다.

text와 table을 input으로 받아 하나의 입력 시퀀스를 생성하고, 시퀀스 내 특정 비율(ex. 15%)의 토큰에 총 2가지 방식의 masking을 수행합니다.

<img src="imgs/tapas_masking.png" width="1000" height="600">


1. **Whole Word Masking**: 랜덤으로 선택된 단어의 모든 토큰을 masking

2. **Whole Cell Masking**: 랜덤으로 선택된 table cell의 모든 토큰을 masking

<br/>

이렇게 2가지 방식의 masking을 통해 위 그림과 같은 Table MLM의 input을 생성하게 됩니다.

지금부터 한단계씩 input 형태를 갖춰가는 과정을 살펴보겠습니다.

### 1.1. `TableTokenizer` 정의

우선, table 전용 토크나이저인 `TapasTokenizer`부터 불러오겠습니다.

In [8]:
from transformers import TapasTokenizer

tapas_tokenizer = TapasTokenizer.from_pretrained(args.vocab_model_name)
tapas_tokenizer.save_pretrained(args.tok_path)

tokenizer = TapasTokenizer.from_pretrained(args.tok_path)

In [9]:
tokenizer.vocab['[PAD]']

0

In [10]:
len(tokenizer) #vocab 파일에 명시되어있지는 않지만 special token으로는 포함되어 있다!

32001


우리는 이 `TapasTokenizer`를 그대로 쓰지 않고, Table-aware MLM 학습에 맞게 변형해서 활용할 것입니다.

이를 위해, `TapasTokenizer` class를 상속받아 필요한 method를 직접 정의하는 작업이 필요합니다.

아래처럼 말이죠!

> **❣ 주의 ❣**
> 
> _transformers 라이브러리의 버전을 꼭 확인하세요!! 버전 마다 구현체가 조금씩 다르기 때문에, 설치된 버전과 다른 버전에서 정의된 변수나 메소드를 불러올 시 에러가 발생하게 됩니다! 현재 설치되어 있는 버전에 맞는 소스코드를 참고하세요! 참고로 제가 사용한 버전은 `transformers==4.11.3` 입니다._

In [11]:
## tokenizer_utils.py에 위치할 것

def create_token_word_idx_list(word_list):
    # token 별로 몇번째 word에 속해있는지 파악하기 위한 index 리스트 생성
    n = 0
    word_idx = []
    for r in range(len(word_list)):
        toks = tokenizer.tokenize(word_list[r])
        idxs = []
        for _ in range(len(toks)):
            n += 1
            idxs.append(n)
        word_idx.append(idxs)

    assert len(word_idx) == len(word_list)
    
    return word_idx

In [12]:
class TableTokenizer(TapasTokenizer):
    def __init__(
        self,
        **kwargs
    ):
        super().__init__(**kwargs)
        
        
    def query_truncate(self, query, max_query_len):
        """
        TapasTokenizer에는 max_query_len에 맞게 query를 truncate하는 기능이 없습니다.
        query token의 길이가 max_query_len을 넘겼을 때나, min_query_len보다 짧을 때,
        query를 인코딩하지 않고 전부 버립니다.(아래 코드처럼)
            
            def _get_question_tokens(self, query):
                # Tokenizes the query, taking into account the max and min question length.
                query_tokens = self.tokenize(query)
                if self.max_question_length is not None and len(query_tokens) > self.max_question_length:
                    logger.warning("Skipping query as its tokens are longer than the max question length")
                    return "", []
                if self.min_question_length is not None and len(query_tokens) < self.min_question_length:
                    logger.warning("Skipping query as its tokens are shorter than the min question length")
                    return "", []

                return query, query_tokens
        
        왜 전부 버려야 하죠? 조금은 남길 수 있는 거잖아요?
        그래서 query_truncate 메소드를 새로 정의해보았습니다.
        구현 방식에서 더 좋은 아이디어가 있다면 톡~투~미~
        """
        query_tokens = self.tokenize(query)
        
        if len(query_tokens) > max_query_len:
            query_words = query.split()
            query_word_idx = create_token_word_idx_list(query_words)
            for i, query_tokens in enumerate(query_word_idx):
                if max_query_len in query_tokens:
                    end_word_idx = i
            
            query_txt = " ".join(query_words[:end_word_idx])
        
            return query_txt
            
            ### table 길이도 고려해서 max query len 자동 지정하는 방안도 괜찮을거같아요~(나중에)
        else:
            return query
        
        
    def get_idx_features_for_masking(self, table, query, max_length = 512):
        # 1) table 각 cell의 좌표 정보를 비롯하여 query, table을 각각 인코딩한 결과를 받아옵니다.
        table_data, query_ids, table_ids = self.get_coordinates(table, query, max_length)
        
        # 2) query와 table이 몇개의 토큰으로 구성되어 있는지 시퀀스 길이 정보를 받아옵니다.
        len_query, len_table = self.get_length_before_pad(query_ids, table_ids)
        len_info = [len_query, len_table]
        
        # 3) whole word masking(text)을 위해 단어 별 토큰의 위치 정보를 받아옵니다.
        whole_word_idxs = self.get_whole_word_info(query_ids)
        
        # 4) whole cell masking(table)을 위해 cell 별 토큰의 위치 정보를 받아옵니다.
        whole_cell_idxs = self.get_whole_cell_info(len_query, table_data)
        
        whole_word_idxs.extend(whole_cell_idxs)
        # print(whole_word_idxs, len_info)
        
        return whole_word_idxs, len_info, query_ids, table_ids
            
    # 1) table 각 cell의 좌표 정보를 비롯하여 query, table을 각각 인코딩한 결과를 받아옵니다.
    def get_coordinates(self, table, query, max_length = 512, truncation="drop_rows_to_fit"):
        ## truncation : DROP_ROWS_TO_FIT = "drop_rows_to_fit",  DO_NOT_TRUNCATE = "do_not_truncate"
        
        ## https://github.com/huggingface/transformers/blob/v4.11.3/src/transformers/models/tapas/tokenization_tapas.py#L1039
        table_tokens = self._tokenize_table(table)
        query_tokens = self.tokenize(query) # query, query_tokens = self._get_question_tokens(query) #(4.xx.x 버전)
        
        ## https://github.com/huggingface/transformers/blob/v4.11.3/src/transformers/models/tapas/tokenization_tapas.py#L1138
        num_rows = self._get_num_rows(table, truncation != "do_not_truncate") #row idx는 0부터 시작
        num_columns = self._get_num_columns(table) #col idx는 1부터 시작
        _, _, num_tokens = self._get_table_boundaries(table_tokens)

        if truncation != "do_not_truncate":
            num_rows, num_tokens = self._get_truncated_table_rows(
                query_tokens, table_tokens, num_rows, num_columns, max_length, truncation_strategy=truncation
            )
        
        table_data = list(self._get_table_values(table_tokens, num_columns, num_rows, num_tokens))
        # [TokenValue(token='층', column_id=1, row_id=0),
        #  TokenValue(token='##수', column_id=1, row_id=0),
        #  TokenValue(token='시설', column_id=2, row_id=0),
        #  TokenValue(token='비', column_id=3, row_id=0),
        #  TokenValue(token='##고', column_id=3, row_id=0),
        # ...
        
        query_ids = self.convert_tokens_to_ids(query_tokens)
        
        table_ids = list(zip(*table_data))[0] if len(table_data) > 0 else list(zip(*table_data))
        table_ids = self.convert_tokens_to_ids(list(table_ids))
    
        return table_data, query_ids, table_ids
    
    
    # 2) query와 table이 몇개의 토큰으로 구성되어 있는지 시퀀스 길이 정보를 받아옵니다.
    def get_length_before_pad(self, query_data, table_data):
        return len(query_data), len(table_data)
        
        
    # 3) whole word masking(text)을 위해 단어 별 토큰의 위치 정보를 받아옵니다.
    def get_whole_word_info(self, query_ids):
        '''whole word index information for text masking'''
        ref_tokens = []
        for n, token_idx in enumerate(query_ids):
            token = tokenizer._convert_id_to_token(token_idx)
            ref_tokens.append(token)
            
        cand_indexes = []
        for (i, token) in enumerate(ref_tokens):
            if len(cand_indexes) >= 1 and token.startswith("##"):
                cand_indexes[-1].append(i+1)
            else:
                cand_indexes.append([i+1])
        
        return cand_indexes
    
    
    # 4) whole cell masking(table)을 위해 cell 별 토큰의 위치 정보를 받아옵니다.
    def get_whole_cell_info(self, len_query, table_cell_info):
        '''whole cell index information fir table masking'''
        idx_order = len_query + 2
        total_cell_cands, one_cell = [], []
        start_row = 0
        start_col = 1
        
        for tok_idx, tok_info in enumerate(table_cell_info):
            if tok_idx == len(table_cell_info) - 1:
                one_cell.append(idx_order+tok_idx)
                total_cell_cands.append(one_cell)
            else:
                if tok_info.row_id == start_row:
                    if tok_info.column_id == start_col:
                        one_cell.append(idx_order+tok_idx)
                    else:
                        total_cell_cands.append(one_cell)
                        one_cell = []
                        start_col += 1
                        one_cell.append(idx_order+tok_idx)
                else:
                    total_cell_cands.append(one_cell)
                    one_cell = []
                    start_row += 1
                    start_col = 1
                    one_cell.append(idx_order+tok_idx)
        
        return total_cell_cands
    

새롭게 정의된 TableTokenizer가 어떤 식으로 작동하는지 아래 예시를 통해 살펴보겠습니다!

In [13]:
tokenizer = TableTokenizer.from_pretrained(args.tok_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'TapasTokenizer'. 
The class this function is called from is 'TableTokenizer'.


In [14]:
ex_text = "N서울타워의 층수는 P0, P1, P2, EZ, T1, T2, T3, T5로 총 8개 층으로 되어있다. P는 플라자의 약자이며 출입구와 약간의 상가로 구성되어있다. EZ는 익스프레스 존의 약자이며 흰색 기둥부분을 가리킨다. T는 타워의 약자이며 전망대와 스낵코너, 그리고 식당으로 구성되어 있다. 지하 1층에서 지상 5층까지는 서울타워 플라자의 시설이 있고 5층부터 타워 1층에서 타워 5층까지는 N서울타워의 시설이 있다. 남산 케이블카도 유명하다."
ex_text

'N서울타워의 층수는 P0, P1, P2, EZ, T1, T2, T3, T5로 총 8개 층으로 되어있다. P는 플라자의 약자이며 출입구와 약간의 상가로 구성되어있다. EZ는 익스프레스 존의 약자이며 흰색 기둥부분을 가리킨다. T는 타워의 약자이며 전망대와 스낵코너, 그리고 식당으로 구성되어 있다. 지하 1층에서 지상 5층까지는 서울타워 플라자의 시설이 있고 5층부터 타워 1층에서 타워 5층까지는 N서울타워의 시설이 있다. 남산 케이블카도 유명하다.'

In [15]:
ex_table = [["층수", "시설", "비고"],
["", "엔그릴/기계실", "양식당 '엔그릴'이며 이곳에서는 개성과 인천의 관측도 가능하다..."],
["타워 6층", "N칼국수,전망대", "... 휴전선까지 관측 가능하다"],
["타워 5층", "", "디지털 전망대와 상행 엘리베이터가 있다."],
["타워 4층", "전망대, N포토 스튜디오, 하늘 화장실, 투썸커피", "아날로그 전망대와 하행 엘리베이터가 있다."],
["타워 3층", "", "이곳에서는 서울 시내까지만 보인다."],
["타워 2층", "루프테라스, 더 플레이스 다이닝", "루프테라스, 더 플레이스 다이닝이 있다."]]

ex_tbl_df = pd.DataFrame(ex_table)
ex_tbl_df = ex_tbl_df.rename(columns=ex_tbl_df.iloc[0])
ex_tbl_df = ex_tbl_df.drop(ex_tbl_df.index[0])
ex_tbl_df.reset_index(drop=True, inplace=True)
ex_tbl_df = ex_tbl_df.astype('str')

for row_index, row in ex_tbl_df.iterrows():
    for col_index, cell in enumerate(row):
        #print(row_index, col_index, ex_tbl_df.iloc[row_index, col_index])
        if ex_tbl_df.iloc[row_index, col_index] == "":
            ex_tbl_df.iloc[row_index, col_index] = "nan"
        else:
            continue

ex_tbl_df

,층수,시설,비고
0,nan,엔그릴/기계실,양식당 '엔그릴'이며 이곳에서는 개성과 인천의 관측도 가능하다...
1,타워 6층,"N칼국수,전망대",... 휴전선까지 관측 가능하다
2,타워 5층,nan,디지털 전망대와 상행 엘리베이터가 있다.
3,타워 4층,"전망대, N포토 스튜디오, 하늘 화장실, 투썸커피",아날로그 전망대와 하행 엘리베이터가 있다.
4,타워 3층,nan,이곳에서는 서울 시내까지만 보인다.
5,타워 2층,"루프테라스, 더 플레이스 다이닝","루프테라스, 더 플레이스 다이닝이 있다."


In [21]:
ex_toks = tokenizer(table=ex_tbl_df, queries=ex_text, max_length=300, padding=True)
ex_toks

{'input_ids': [2, 50, 28750, 2256, 2667, 2079, 1688, 2113, 2259, 52, 2082, 16, 52, 2083, 16, 52, 2302, 16, 41, 2611, 16, 56, 2083, 16, 56, 2302, 16, 56, 2195, 16, 56, 2049, 2200, 1668, 28, 2019, 1688, 6233, 859, 2051, 2689, 2062, 18, 52, 2259, 16597, 2079, 9383, 2052, 2307, 18455, 2522, 4943, 2079, 6682, 2200, 3896, 2496, 2051, 2689, 2062, 18, 41, 2611, 2259, 24284, 1554, 2079, 9383, 2052, 2307, 12003, 9856, 12547, 2069, 16519, 18, 56, 2259, 8203, 2079, 9383, 2052, 2307, 14822, 2522, 1, 16, 3673, 5499, 6233, 3896, 2496, 2051, 1513, 2062, 18, 4670, 21, 2624, 27135, 5377, 25, 2624, 2299, 2118, 2259, 3671, 2256, 2667, 16597, 2079, 3953, 2052, 1513, 2088, 25, 2624, 3797, 8203, 21, 2624, 27135, 8203, 25, 2624, 2299, 2118, 2259, 50, 28750, 2256, 2667, 2079, 3953, 2052, 1513, 2062, 18, 12103, 15879, 2119, 4455, 2205, 2062, 18, 3, 1688, 2113, 3953, 1187, 2088, 32000, 1423, 2029, 2388, 19, 5276, 2477, 6277, 2481, 11, 1423, 2029, 2388, 11, 1504, 2307, 4441, 27135, 2259, 5879, 2145, 4068, 2079, 6

- `attention_mask`: table&text 토큰=1, padding=0

- `token_type_ids`: TapasTokenizer에는 테이블 구조를 반영하기 위한 7개의 token type id가 추가되어 있습니다. 각 id에 대한 설명은 아래 Tapas 공식 문서를 발췌한 부분을 참고해주세요!

    1. `segment_ids`: indicate **whether a token belongs to the question (0) or the table (1)**. 0 for special tokens and padding.
    2. `column_ids`: indicate to **which column of the table** a token belongs (starting from 1). Is 0 for all question tokens, special tokens and padding.
    3. `row_ids`: indicate to **which row of the table** a token belongs (starting from 1). Is 0 for all question tokens, special tokens and padding. Tokens of column headers are also 0.
    4. `prev_labels`: indicate **whether a token was (part of) an answer to the previous question** (1) or not (0). Useful in a conversational setup (such as SQA dataset).
    5. `column_ranks`: indicate the **rank of a table token relative to a column**, **if applicable**. For example, if you have a column "number of movies" with values 87, 53 and 69, then the column ranks of these tokens are 3, 1 and 2 respectively. 0 for all question tokens, special tokens and padding.
    6. `inv_column_ranks`: indicate the **inverse rank** of a table token relative to a column, **if applicable**. For example, if you have a column "number of movies" with values 87, 53 and 69, then the inverse column ranks of these tokens are 1, 3 and 2 respectively. 0 for all question tokens, special tokens and padding.
    7. `numeric_relations`: indicate numeric relations between the question and the tokens of the table. 0 for all question tokens, special tokens and padding.

In [22]:
tokenizer.decode(ex_toks['input_ids'])

"[CLS] N서울타워의 층수는 P0, P1, P2, EZ, T1, T2, T3, T5로 총 8개 층으로 되어있다. P는 플라자의 약자이며 출입구와 약간의 상가로 구성되어있다. EZ는 익스프레스 존의 약자이며 흰색 기둥부분을 가리킨다. T는 타워의 약자이며 전망대와 [UNK], 그리고 식당으로 구성되어 있다. 지하 1층에서 지상 5층까지는 서울타워 플라자의 시설이 있고 5층부터 타워 1층에서 타워 5층까지는 N서울타워의 시설이 있다. 남산 케이블카도 유명하다. [SEP] 층수 시설 비고 [EMPTY] 엔그릴 / 기계실 양식당'엔그릴'이며 이곳에서는 개성과 인천의 관측도 가능하다... 타워 6층 N칼국수, 전망대... 휴전선까지 관측 가능하다 타워 5층 [EMPTY] 디지털 전망대와 상행 엘리베이터가 있다. 타워 4층 전망대, N포토 스튜디오, 하늘 화장실, 투썸커피 아날로그 전망대와 하행 엘리베이터가 있다. 타워 3층 [EMPTY] 이곳에서는 서울 시내까지만 보인다. 타워 2층 루프테라스, 더 플레이스 다이닝 루프테라스, 더 플레이스 다이닝이 있다. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [23]:
idx_info, len_info, query_ids, table_ids = tokenizer.get_idx_features_for_masking(table=ex_tbl_df, query=ex_text, max_length=300)

In [24]:
print(idx_info)

[[1, 2, 3, 4, 5], [6, 7, 8], [9, 10], [11], [12, 13], [14], [15, 16], [17], [18, 19], [20], [21, 22], [23], [24, 25], [26], [27, 28], [29], [30, 31, 32], [33], [34, 35], [36, 37], [38, 39, 40, 41], [42], [43, 44], [45, 46], [47, 48, 49], [50, 51], [52, 53], [54, 55], [56, 57, 58, 59, 60], [61], [62, 63, 64], [65], [66, 67], [68, 69, 70], [71], [72, 73, 74], [75], [76], [77, 78], [79, 80], [81, 82, 83], [84, 85], [86], [87], [88], [89, 90], [91, 92, 93], [94, 95], [96], [97], [98, 99, 100], [101], [102, 103, 104, 105, 106], [107, 108, 109], [110, 111], [112, 113], [114, 115], [116, 117, 118], [119], [120, 121, 122], [123], [124, 125, 126, 127, 128], [129, 130, 131, 132, 133], [134, 135], [136, 137], [138], [139], [140, 141], [142, 143, 144], [145], [147, 148], [149], [150, 151], [152], [153, 154, 155, 156, 157, 158], [159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182], [183, 184, 185], [186, 187, 188, 189, 190], [191, 

각 하위 리스트들은 단어, cell에 속한 토큰들의 위치 정보를 포함합니다.

예를 들어, query(text)의 첫번째 단어(`[1, 2, 3, 4, 5]`)는 첫번째 토큰부터 다섯번째 토큰으로 구성되어 있습니다.

참고로 `[CLS]` 토큰에 해당하는 0번 인덱스는 이 리스트에 포함되어 있지 않으며, 마찬가지로 text와 table을 분리하는 `[SEP]` 토큰 역시 제외된 상태입니다.

중간에 토큰의 위치 인덱스가 비어있는 지점(위 예시에서는 146번 인덱스)가 text에서 table로 넘어가는 `[SEP]` 토큰의 위치라고 보시면 됩니다.

<br/>

또한, 아래와 같이 query(text)와 table의 길이 정보, 인코딩 결과가 함께 제공됩니다.

In [25]:
print(len_info)

[145, 130]


In [26]:
print(query_ids)

[50, 28750, 2256, 2667, 2079, 1688, 2113, 2259, 52, 2082, 16, 52, 2083, 16, 52, 2302, 16, 41, 2611, 16, 56, 2083, 16, 56, 2302, 16, 56, 2195, 16, 56, 2049, 2200, 1668, 28, 2019, 1688, 6233, 859, 2051, 2689, 2062, 18, 52, 2259, 16597, 2079, 9383, 2052, 2307, 18455, 2522, 4943, 2079, 6682, 2200, 3896, 2496, 2051, 2689, 2062, 18, 41, 2611, 2259, 24284, 1554, 2079, 9383, 2052, 2307, 12003, 9856, 12547, 2069, 16519, 18, 56, 2259, 8203, 2079, 9383, 2052, 2307, 14822, 2522, 1, 16, 3673, 5499, 6233, 3896, 2496, 2051, 1513, 2062, 18, 4670, 21, 2624, 27135, 5377, 25, 2624, 2299, 2118, 2259, 3671, 2256, 2667, 16597, 2079, 3953, 2052, 1513, 2088, 25, 2624, 3797, 8203, 21, 2624, 27135, 8203, 25, 2624, 2299, 2118, 2259, 50, 28750, 2256, 2667, 2079, 3953, 2052, 1513, 2062, 18, 12103, 15879, 2119, 4455, 2205, 2062, 18]


In [27]:
print(table_ids)

[1688, 2113, 3953, 1187, 2088, 32000, 1423, 2029, 2388, 19, 5276, 2477, 6277, 2481, 11, 1423, 2029, 2388, 11, 1504, 2307, 4441, 27135, 2259, 5879, 2145, 4068, 2079, 6541, 2119, 3662, 2205, 2062, 18, 18, 18, 8203, 26, 2624, 50, 2600, 9473, 16, 14822, 18, 18, 18, 31099, 2299, 2118, 6541, 3662, 2205, 2062, 8203, 25, 2624, 32000, 5476, 14822, 2522, 1242, 2375, 10874, 2116, 1513, 2062, 18, 8203, 24, 2624, 14822, 16, 50, 2208, 2386, 9238, 16, 4573, 7047, 16, 1801, 3428, 20468, 16834, 14822, 2522, 1889, 2375, 10874, 2116, 1513, 2062, 18, 8203, 23, 2624, 32000, 4441, 27135, 2259, 3671, 6011, 2299, 3683, 4090, 18, 8203, 22, 2624, 19283, 2201, 5822, 16, 831, 18312, 5970, 2944, 19283, 2201, 5822, 16, 831, 18312, 5970, 2944, 2052, 1513, 2062, 18]


### 1.2. `TableDataset` 정의

위에서 커스터마이징한 Table Tokenizer를 활용하여 학습에 필요한 TableDataset을 정의하겠습니다.

In [25]:
## dataset_utils.py에 위치할 것

def none_to_nan(name):
    return "nan" if name == None else name

def del_row_or_col(df, row_del_ratio, col_del_ratio):
    """nan 개수가 일정 비율 이상이면 해당 column/row 전부 삭제하기 위한 함수입니다."""
    
    n_r = df.shape[0]
    n_c = df.shape[1]
    
    c_cand_del = []
    for i_c in range(n_c):
        n_col_nan = df.iloc[:,i_c].isnull().sum()
        if n_col_nan > 0:
            col_nan_ratio = n_col_nan/n_r
            if col_nan_ratio > col_del_ratio:
                # print(f"col {i_c}({df.columns[i_c]}) insnull ratio: {col_nan_ratio}")
                c_cand_del.append(df.columns[i_c])
    
    r_cand_del = []
    for i_r in range(n_r):
        n_row_nan = df.iloc[i_r,:].isnull().sum()
        if n_row_nan > 0:
            row_nan_ratio = n_row_nan/n_c
            if row_nan_ratio > row_del_ratio:
                # print(f"row {i_r}({df.index[i_r]}) insnull ratio: {row_nan_ratio}")
                r_cand_del.append(df.index[i_r])    
    
    # del nan col
    df = df.drop(c_cand_del, axis=1)
    # del nan row
    df = df.drop(r_cand_del)
    
    # 다 지운 담에는 np.nan -> "nan"으로 변경해서 [EMPTY] 토큰으로 인식되도록!
    df = df.fillna('nan')
    
    return df

In [31]:
import copy
import pandas as pd
import random

import torch
from torch.utils.data import Dataset

from transformers import TapasTokenizer
# from table_tokenizer import TableTokenizer


class TableDataset(Dataset):
    def __init__(self, data, args=None, tokenizer=None):
        self.data = data
        self.args = args
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, data_idx):
        data_dict = self.data[data_idx]
        text = data_dict['Description']
        table = self.convert_list_to_df(data_dict['TBL'], self.args.row_del_ratio, self.args.col_del_ratio)
        
        real_text = self.tokenizer.query_truncate(text, self.args.max_query_len) ###max_query_len
        
        idx_info, len_info, _, _ = self.tokenizer.get_idx_features_for_masking(table=table, query=real_text, max_length=self.args.max_seq_len)
        
        inputs = self.tokenizer(table=table, queries=real_text, padding=True, max_length = self.args.max_seq_len, truncation=True)
        # idx_info, len_info, _, _ = self.tokenizer.get_idx_features_for_masking(table=table, query=text)
        # ## [[1, 2, 3, 4, 5], [6, 7, 8], [9, 10], [11], [12, 13], [14], [15, 16], ...]
        
        idx_info_copy = copy.deepcopy(idx_info)
        
        input_ids, labels = self.create_features_for_mlm(inputs['input_ids'], idx_info, len_info)
        
        return [input_ids, inputs['attention_mask'], inputs['token_type_ids'], labels, idx_info_copy]
    
    
    def convert_list_to_df(self, table_list, row_del_ratio, col_del_ratio):
        """
        이 메소드는 list로 주어진 table input을 받아 pandas dataframe 형식으로 변환 및 간단한 전처리를 수행합니다.
        구체적인 프로세스는 주석을 참고하세요!
        """
        # 1. list -> dataframe
        tbl_df = pd.DataFrame(table_list)
        # 2. 첫번째 행은 column 이름!
        tbl_df = tbl_df.rename(columns=tbl_df.iloc[0])
        # 3. column 이름으로 올려놨으니 첫번째 행은 지우기!
        tbl_df = tbl_df.drop(tbl_df.index[0])
        # 4. row index 초기화!
        tbl_df.reset_index(drop=True, inplace=True)
        # 5. 데이터 타입 문자열로 변환해야 None -> "None"으로 바뀌고, tokenizing 가능!
        tbl_df = tbl_df.astype('str')

        # 6. row, column 별로 돌면서 비어있는 값은 모두 결측치(np.nan)로 변환 -> 결측치 많은 행, 열은 제거할 것이기 때문!
        for row_index, row in tbl_df.iterrows():
            for col_index, cell in enumerate(row):
                if tbl_df.iloc[row_index, col_index].strip() == "" or tbl_df.iloc[row_index, col_index] == "None":
                    tbl_df.iloc[row_index, col_index] = np.nan ###
                else:
                    continue

        # 7. 결측치가 너무 많은 row와 column은 제거!
        tbl_df = del_row_or_col(tbl_df, row_del_ratio, col_del_ratio)

        # 8. column 이름이 None인 경우 Nonetype error 방지를 위해 nan으로 rename!
        tbl_df.columns = [none_to_nan(col) for col in list(tbl_df.columns)]

        return tbl_df
    
    
    def create_features_for_mlm(self, org_input_ids, idx_info, len_info):
        
        # masking ref: https://github.com/huggingface/transformers/blob/v4.11.3/src/transformers/data/data_collator.py#L924
        
        len_query_table = sum(len_info)
    
        num_to_predict = min(self.args.max_seq_len, max(1, int(round(len_query_table * self.args.mlm_prob))))

        random.shuffle(idx_info) #랜덤으로 섞고 앞에서부터 차례로 masking할 토큰수 만큼 채움
        ## [[1, 2, 3, 4, 5], [6, 7, 8], [9, 10], [11], [12, 13], [14], [15, 16], [17], ...] -> [[94, 95], [11], [62, 63, 64], ...]

        masked_lms = []
        covered_indexes = set()
        for index_set in idx_info:
            if len(masked_lms) >= num_to_predict:
                break
            # If adding a whole-word mask would exceed the maximum number of predictions, then just skip this candidate.
            if len(masked_lms) + len(index_set) > num_to_predict:
                continue
            is_any_index_covered = False
            for index in index_set:
                if index in covered_indexes:
                    is_any_index_covered = True
                    break
            if is_any_index_covered:
                continue
            for index in index_set:
                covered_indexes.add(index)
                masked_lms.append(index)

        assert len(covered_indexes) == len(masked_lms)
        mask_labels = [1 if i in covered_indexes else 0 for i in range(len_query_table + 2)] #+2: [CLS]와 [SEP] 개수도 포함!!

        for _ in range(self.args.max_seq_len - len(mask_labels)):
            mask_labels.append(self.tokenizer.pad_token_id) # 0


        # 1. labels
        inputs = torch.Tensor(org_input_ids)
        labels = inputs.clone()

        masked_indices = torch.Tensor(mask_labels).bool()

        labels[~masked_indices] = -100 #masking된 토큰을 제외하고서는 loss 연산에서 제외
        
        real_labels = labels.tolist()
        
        real_labels = [int(lab) for lab in real_labels]

        # 2. input_ids
        input_ids = inputs.tolist()
        for mask_pos in masked_lms:
            input_ids[mask_pos] = self.tokenizer.mask_token_id # 4
            
        input_ids = [int(inp) for inp in input_ids]

        return input_ids, real_labels
        

### 1.3. `table collate 함수` 및 `DatoLoader` 정의

In [32]:
import torch

def table_collate_fn(batch):
    
    features = {
        'input_ids': torch.LongTensor([sample[0] for sample in batch]),
        'attention_mask': torch.LongTensor([sample[1] for sample in batch]),
        'token_type_ids': torch.LongTensor([sample[2] for sample in batch]),
        'labels': torch.LongTensor([sample[3] for sample in batch]),
        'offsets': [sample[4] for sample in batch]
    }
    
    return features

In [33]:
from torch.utils.data import DataLoader

train_data = sample_data['data'][:170]
valid_data = sample_data['data'][170:]

train_dataset = TableDataset(train_data, args, tokenizer)
valid_dataset = TableDataset(valid_data, args, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=table_collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=args.batch_size, collate_fn=table_collate_fn)

아래 cell은 batch 예시를 출력해보기 위해 선언한 가상의 dataloader입니다.

실제 학습 때는 실행하지 않으셔도 됩니다.

In [34]:
ex_dataloader = DataLoader(train_dataset, batch_size=2, collate_fn=table_collate_fn)

batch_ex = next(iter(ex_dataloader))
print(batch_ex)

{'input_ids': tensor([[   2,  170, 1376,  ...,    0,    0,    0],
        [   2,   14,  809,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]]), 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]]), 'offsets': [[[1], [2, 3], [4], [5], [6], [7], [8], [9, 10, 11], [12, 13, 14], [15], [16], [17, 18], [19, 20], [21], [22, 23], [24, 25], [26], [27, 28], [29], [31, 32, 33, 34, 35, 36, 37, 38, 39, 40], [41, 

In [39]:
print(f"1. input ids: \n{batch_ex['input_ids'][0]}")
print("-----------------------------------------")
print(f"2. labels: \n{batch_ex['labels'][0]}")
print("-----------------------------------------")

1. input ids: 
tensor([    2,   170,  1376,  2320,  8347,   171,    12,     4,    30, 26384,
         2054,  4704, 28495, 15869,  2041,    13,   793, 17591,  2615,  9296,
         2170,     4,     4,     4,  6837,  2897,  9296,  3771, 28674,    18,
            3,  1376,  2320,  8347,  2106,  2008,  7088,  4704, 28495, 15869,
         2041,  1376,  2320,  8347,  2106,  2008,  7088,  4704, 28495, 15869,
         2041,  3871,     4,     4,     4,     4,  4734, 10073,  1050,  2506,
        16555,  2548,  4358, 22330, 17194,  4152, 14113, 13429,  2029,  2234,
         3193,  2059,  2269,  2255, 12300,     4,     4,     4,     4,     4,
            4,     4,     4, 28157, 27357,  1883,  2294,  2255, 11398,  1883,
         2294,  2255,  5551,  6837,  2210,  4840,  2440,  3718,  2429,  3718,
         2210,  4840,  2440,  3718,  2429,  4136,  2210,  4840,  2440,  3718,
         2429,  3912,  2210,  3641,  7815,  2377,  3728,     4,     4,  4686,
         4612,     0,     0,     0,     0,     0,

`input ids`에서 masking된 token은 masking 인덱스인 4를 부여받고, `labels`에서는 maksing된 token 외 모든 토큰들이 전부 -100의 인덱스를 부여받은 것을 확인할 수 있습니다.

이제 MLM 학습을 위한 데이터 구성은 모두 마쳤습니다!

모델링 파트로 넘어가보아요~!

---

## 2. 모델링

Tapas는 embedding을 제외하고서 BERT와 동일한 구조로 구성되어 있으며, 코드 역시 BERT 구현체와 상당히 유사합니다.

따라서, huggingface의 `BertModel` 계열 소스코드를을 접해보신 분들이라면 익숙하실 겁니다.

Tapas에 MLM을 추가한 구조는 `TapasForMaskedLM`을 통해 손쉽게 불러올 수 있습니다.

그 전에 모델의 정보를 담은 `TapasConfig`를 호출하겠습니다.

### 2.1. `TapasConfig` 수정

> _"그냥 모델 가져다 쓰면 된다면서요. 왜 config를 수정하는거죠?"_

TapasConfig는 기준 언어가 영어로 맞춰져 있어서 모델 초반 Embedding Layer를 관할하는 `vocab_size`가 `bert-base-uncased`의 vocab 개수인 30522로 설정되어 있습니다.

In [40]:
from transformers import TapasConfig
config = TapasConfig()

In [41]:
config.vocab_size

30522

우리는 한국어 table mrc를 위한 tapas 모델을 구축해야 하기 때문에, 일전에 `TableTokenizer`를 로드할 때 사용했던 `klue/bert-base`에 맞는 vocab_size를 새로 정의해주어야 합니다.

`TapasTokenizer`에는 special token인 `[EMPTY]`가 추가로 부여되어 있는 관계로, 기존 `klue/bert-base`의 vocab size(32000)에서 1을 추가한 32001로 수정하겠습니다!

In [42]:
config.vocab_size = 32001

In [43]:
config

TapasConfig {
  "aggregation_labels": null,
  "aggregation_loss_weight": 1.0,
  "aggregation_temperature": 1.0,
  "allow_empty_column_selection": false,
  "answer_loss_cutoff": null,
  "answer_loss_importance": 1.0,
  "attention_probs_dropout_prob": 0.1,
  "average_approximation_function": "ratio",
  "average_logits_per_cell": false,
  "cell_selection_preference": null,
  "disable_per_token_loss": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "huber_loss_delta": null,
  "init_cell_selection_weights_to_zero": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_num_columns": 32,
  "max_num_rows": 64,
  "max_position_embeddings": 1024,
  "model_type": "tapas",
  "no_aggregation_label_index": null,
  "num_aggregation_labels": 0,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "positive_label_weight": 10.0,
  "reset_position_index_per_cell": true,
  "select_one_column": true,

맨 아래 `vocab_size`를 보니 잘 수정이 되었네요.

- 바로 아래 `2.2. TapasForMaskedLM 소환`에서 model 구조를 출력한 셀을 보면 Word Embedding layer의 input 차원 역시 32001로 바뀐 것을 확인할 수 있습니다!

이제 모델을 불러옵시다.

### 2.2. `TapasForMaskedLM` 소환

위에서 수정한 Tapas config를 입력 변수로 넣어 미리 잘 구축된 Tapas MLM 모델을 불러오겠습니다.

In [44]:
from transformers import TapasForMaskedLM

model = TapasForMaskedLM(config)

In [45]:
if n_gpu > 1:
    model = torch.nn.DataParallel(model).to(args.device)
    
else:
    model = model.to(args.device)

앞서 말씀드렸 듯이 MLM은 맨 마지막 classifier에서 vocab size 만큼의 output label 수를 가지기 때문에 발생되는 파라미터 수가 굉장히 큽니다.

따라서, 메모리 용량이 큰 single GPU나 여러 개의 GPU가 할당된 서버가 아닌 이상, 학습을 수행하기 어렵습니다.

제 경우에는 multi GPU인 `Titan-RTX-4way`(24GB*4)를 사용하였습니다.

multi-gpu를 활용하는 경우, 분산 학습을 위해 모델은 DataParallel이라는 객체에 묶이게 됩니다.

아래 model 구조를 참고해주세요!

In [46]:
model

DataParallel(
  (module): TapasForMaskedLM(
    (tapas): TapasModel(
      (embeddings): TapasEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(1024, 768)
        (token_type_embeddings_0): Embedding(3, 768)
        (token_type_embeddings_1): Embedding(256, 768)
        (token_type_embeddings_2): Embedding(256, 768)
        (token_type_embeddings_3): Embedding(2, 768)
        (token_type_embeddings_4): Embedding(256, 768)
        (token_type_embeddings_5): Embedding(256, 768)
        (token_type_embeddings_6): Embedding(10, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TapasEncoder(
        (layer): ModuleList(
          (0): TapasLayer(
            (attention): TapasAttention(
              (self): TapasSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (k

우리가 사전학습을 잘 수행하고서 풀어야 할 downstream task는 결국 Table Question-Answering입니다.

Fine-tuning(Table QA 학습)을 할 때, Pre-training에 쓰였던 모든 파라미터가 다 필요하지는 않습니다.

- query와 table 각 토큰들의 Embedding을 수행하고 representation을 생성하는 `TapasModel`만이 fine-tuning 모델 구축에 활용됩니다.

In [1]:
# 이 cell을 실행하시면 TapasModel의 구조만 따로 확인할 수 있씁니다.
model.module.tapas

따라서, 모델 학습 과정 중에 저장할 파라미터는 `torch.save(model.module.tapas.state_dict(), output_dir)`와 같은 형식으로 정의됩니다.

- 이에 대한 코드는 아래 `3. 학습` 파트의 `train` 함수에서 (*) 부분을 보시면 됩니다!

---

## 3. 학습

모델을 정의했으니, 이제 학습과 평가에 필요한 train, evaluate 함수를 정의하고, 실제 학습을 수행할 단계입니다.

학습 과정 중 기록되는 log는 `wandb` 플랫폼에서 시각화 되도록 설계했습니다.

### 3.1. 학습 준비

#### 3.1.1. train, eval 함수 정의

In [48]:
def train(args, model, train_dataloader, valid_dataloader, optimizer, lr_scheduler):
    
    global_step = 0
    best_loss = 100

    wandb.init(
        project=args.wandb_project,
        name=args.wandb_name,
        entity=args.wandb_entity
    )
    
    wandb.config.update(
        {
            "epochs": args.epoch,
            "batch_size": args.batch_size,
            "learning_rate": args.learning_rate,
        }
    )
    wandb.watch(model, log="all")

    for epoch in tqdm(range(1, args.epoch+1)):
        
        model.train()
        
        losses = 0
        for batch_idx, batch in enumerate(train_dataloader):
            
            inputs = {
                'input_ids': batch['input_ids'].to(args.device),
                'attention_mask': batch['attention_mask'].to(args.device),
                'token_type_ids': batch['token_type_ids'].to(args.device),
                'labels': batch['labels'].to(args.device),
            }
            
            output = model(**inputs)

            loss = output.loss.mean()
            losses += loss.item()

            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            global_step += 1

            
            if global_step % args.eval_step == 0:
                
                eval_loss = evaluate(
                    args, model,
                    valid_dataloader
                )
                
                # train logging
                wandb.log({
                    'train_mlm_loss': loss.item(),
                    'eval_mlm_loss': eval_loss
                })
                

                if eval_loss < best_loss:
                    best_loss = eval_loss
                    output_dir = os.path.join(args.output_dir, "best_tapas_model.pt")
                    
                    # (*) 모델 저장
                    # torch.save(model.tapas.state_dict(), output_dir) #single GPU
                    torch.save(model.module.tapas.state_dict(), output_dir) # multi GPU

        
        epoch_eval_loss = evaluate(args, model,valid_dataloader)
        
        print(f"[Epoch{epoch}] Train mlm loss: {losses/len(train_dataloader)}, Eval mlm loss: {epoch_eval_loss}")

In [50]:
def evaluate(args, model, eval_dataloader):
    
    model.eval()    # close drop out, batch normalization

    eval_loss = 0

    for batch_idx, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            inputs = {
                'input_ids': batch['input_ids'].to(args.device),
                'attention_mask': batch['attention_mask'].to(args.device),
                'token_type_ids': batch['token_type_ids'].to(args.device),
                'labels': batch['labels'].to(args.device),
            }
            
            output = model(**inputs)

            loss = output.loss.mean()

            eval_loss += loss.item()
            
    return eval_loss/len(eval_dataloader)

#### 3.1.2. optimizer, lr_scheduler 정의

In [51]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [
            p
            for n, p in model.named_parameters()
            if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [
            p
            for n, p in model.named_parameters()
            if any(nd in n for nd in no_decay)
        ],
        "weight_decay": args.weight_decay,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

train_step = args.epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
        name=args.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=train_step,
    )

### 3.2. Let's train!

In [21]:
import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

In [52]:
train(args, model, train_dataloader, valid_dataloader, optimizer, lr_scheduler)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: ERROR API key must be 40 characters long, yours was 7
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/go60/.netrc


...

[Epoch3] Train mlm loss: 7.727213035930287, Eval mlm loss: 7.8767523765563965


 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                        | 4/5 [01:37<00:23, 23.92s/it]

[Epoch4] Train mlm loss: 7.543686736713756, Eval mlm loss: 7.736621379852295


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:59<00:00, 23.95s/it]

[Epoch5] Train mlm loss: 7.48064695705067, Eval mlm loss: 7.823406219482422


학습 중간에 출력된 경고 메세지는 DDP가 아닌 DP를 사용해서 그렇습니다.([관련 link](https://github.com/huggingface/transformers/issues/14128))

향후 DDP 버전으로 업데이트할 예정이니 참고해주세요!